###  *Develop an agent that can achieve a specific goal and sub-goal in a maze 10X10 based on RL algorithms.*

In [1]:
# Import the necessary libraries to create the maze and give our agent the ability to randomly making choices

import numpy as np
import random

In [3]:
# Implemetation of the maze 10X10 as indicated in the assignment. 

# 1 represents the walls (unpassable)
# 0 represents an open path (passable)
# S marks the starting point of the agent
# G marks the sub-goal the agent should reach before continuing
# E marks the end goal

maze = np.array([[1,1,1,1,1,1,1,1,1,1],
                 [1,'S',1,0,0,0,1,0,0,1],
                 [1,0,1,0,1,0,1,0,1,1],
                 [1,0,0,0,1,0,0,0,0,1],
                 [1,1,1,0,1,1,1,1,0,1],
                 [1,0,1,'G',0,0,0,1,0,1],
                 [1,0,0,0,1,1,0,0,0,1],
                 [1,1,1,0,1,0,1,1,0,1],
                 [1,0,0,0,0,0,1,'E',0,1],
                 [1,1,1,1,1,1,1,1,1,1]], dtype=object)



# Implementatio of the Q-Table
# Considering the maze above, and the Q-learning algorithm, we need to initiate
# the Q-table. The Q-table include all possible states-actions pairs and their corresponing Q-values.

q_table = np.zeros((10,10,4))   # each states have 4 possible actions (up, down, left, right)
actions = ['up', 'down', 'left', 'right']


# Implementation of the reward system

reward_sub_goal = 10
reward_end_goal = 50
penalty = -1    

# Implementation of the agent's interactions with the environment, the way the agent moves in the maze
# and the way penalties and rewards are given to the agent.

def moves(state, action):
    row, col = state

    if action == 'up':
        next_state = (row - 1, col)
    elif action == 'down':
        next_state = (row + 1, col)
    elif action == 'left':
        next_state = (row, col - 1)
    elif action == 'right':
        next_state = (row, col + 1)
    else:
        next_state = state

# Check if the next state is a wall or out of the maze
    if next_state[0] >= 0 and next_state[0] < 10 and next_state[1] >= 0 and next_state[1] < 10:
        if maze[next_state] != 1:
            if next_state == (5,3):
                return next_state, reward_sub_goal, False   #Sub-goal reached 
            elif next_state == (8,7):
                return next_state, reward_end_goal, True    #End-goal reached
            else:
                return next_state, penalty, False           #No goal reached, keep searching
        else:
            return state, penalty, False                    #Hit a wall, stay in the same state




# Implementation of the starting funtion, in order to restart the agent's position in the maze
# at the end of each episode.

def restarting():
    return start_position